This script will take a MESA stellar evoltion profile and convert it into a format that can be read in as a custom reference state in Rayleigh. You will need the `rayleigh_diagnostics.py`, `reference_tools.py`, and `mesa.py` files. You will also need a suitable MESA profile file, such as `mesa.prof`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as spi
import scipy.integrate as spint
import scipy.signal as spsig
import reference_tools as rt
import sys
import rayleigh_diagnostics as rd
import mesa
%matplotlib inline

In [ ]:
def interp(r, v):
    prad = 10**p.logR[::-1] * mesa.rsol
    return np.interp(r, prad, v[::-1])

Set the `work_dir` variable to the location of the Python files listed above and MESA profile you would like to use.

In [ ]:
work_dir = '/Users/njnelson/Research/Rayleigh/Rayleigh/tmp'
sys.path.append(work_dir)

In [ ]:
p = mesa.profile('profile17.data')

Choose a suitable number of radial grid points. They do not need to be regularly spaced. You should err on the side of high resolution since Rayleigh's Chebyshev domains have very fine grid spacing at the top and bottom of the domain.

In [ ]:
nr = 5000
r0 = 4.55e10 # in cm
r1 = 6.00e10 # in cm
radius = np.linspace(r0, r1, nr)

From the MESA model, Rayleigh will need the density, buoyancy function $ \rho g / C_P$, temperature, vicoscity, thermal diffusion, electrical resistivity (for magnetic cases), heating profile (for cases with $Q \ne 0$), entropy gradient (for cases with reference state advection). Note that MESA radial indicies start at the bottom, while Rayleigh radial indicies start at the top.

In [ ]:
r_MESA = p.rmid*mesa.rsol
density = interp(radius, 10**p.logRho)
temperature = interp(radius, 10**p.logT)
grav = interp(radius, p.grav)
cp = interp(radius, p.cp)
buoy = density * grav / cp
nu = 1e14 * np.ones_like(radius)
kappa = 1e14 * np.ones_like(radius)
eta = 1e14 * np.ones_like(radius)
hprofile = np.zeros_like(radius)

*WARNING* You should be very careful how you think about the entropy gradient when moving from MESA to Rayleigh due to the differing equations of state. We have provided an example that simply takes the density and pressure from MESA and uses Rayleigh's equation of state to compute an entropy gradient, however this will *not* be consistent with the entropy gradient computed by MESA. BE YE WARNED!

For example, the entropy gradient below is not very smooth, which may or may not be a problem for Rayleigh. We can smooth it if desired, as shown below.

In [ ]:
gamma_ideal = 5.0/3.0
dsdr_MESA = -p.cp*(1.0/gamma_ideal*np.gradient(p.logP, r_MESA) - np.gradient(p.logRho,r_MESA))
dsdr = interp(radius, dsdr_MESA)

plt.plot(radius, dsdr, '-b')
plt.plot(r_MESA, dsdr_MESA, '-g')
plt.xlim([r0, r1])
plt.ylim([-2.0e-3, 1.0e-4])

n_win = 201
smooth_win = spsig.hann(n_win)
temp = spsig.convolve(dsdr, smooth_win, mode='same')/sum(smooth_win)
dsdr[(n_win-1)//2:-(n_win-1)//2] = temp[(n_win-1)//2:-(n_win-1)//2]
print(len(dsdr))
plt.plot(radius, dsdr, '-r')

*ANOTHER WARNING* You should be very careful with radiative luminosity and/or nuclear energy generation. There are a number of ways to compute the heating functions you need. For this example, we have chosen to simply compute the luminosity profile needed for flux balance if the convective transport matches the values from MESA.

In [ ]:
q_rad = -np.gradient((p.luminosity - p.conv_L_div_L*p.luminosity)*mesa.solarlum, r_MESA)/(4.0*np.pi*r_MESA**2)
heatingp = interp(radius, q_rad)
luminosity = np.trapz(4.0*np.pi*radius**2*heatingp, radius)
heaitngp = heatingp/luminosity

Plot the density from MESA and the newly interpolated density that will be fed into Rayleigh to make sure they are consistent.

In [ ]:
plt.plot(radius, density)
plt.plot(p.rmid[::-1] * mesa.rsol, 10**p.logRho[::-1])
plt.xlabel('Radius (cm)')
plt.ylabel(r'Desnity (g/cm$^3$)')

Now create the data structure that will be written to a file that Rayleigh can read, and then load in the needed radial functions.

In [ ]:
my_ref = rt.equation_coefficients(radius)

my_ref.set_function(density,'density')
my_ref.set_function(buoy,'buoy')
my_ref.set_function(nu,'nu')
my_ref.set_function(temperature,'temperature')
my_ref.set_function(kappa,'kappa')
my_ref.set_function(hprofile,'heating')
my_ref.set_function(eta,'eta')
my_ref.set_function(dsdr,'ds_dr')

my_ref.set_constant(luminosity,'luminosity')

In [ ]:
file_write='cref_from_MESA.dat'
my_ref.write(file_write)

Now you can use this file to run a Rayleigh model. Once your Rayleigh model has run you can use the ``reference`` and  ``transport`` files to check how your specified reference state looks when transfered into Rayleigh.

In [ ]:
#Once you're run for one time step, set have_run = True
radius1 = radius
gravity = grav
have_run = True
if (have_run):
    cref = rd.ReferenceState()
    ctrans = rd.TransportCoeffs()
    lsun = 1.0
    # Use numpy to calculate logarithmic derivatives (check on what Rayleigh is doing...)
    #d_density_dr = numpy.gradient(density,radius, edge_order=2)
    #dlnrho = d_density_dr/density
    #d2lnrho = numpy.gradient(dlnrho,radius, edge_order=2)

    #dtdr = numpy.gradient(temperature,radius)
    #dlnt = dtdr/temperature
    dr = np.gradient(cref.radius)

    dP = np.gradient(cref.pressure,cref.radius)
    fig, ax = plt.subplots(ncols=3,nrows=3, figsize=(9,3*3))
    # Density variables
    ax[0][0].plot(cref.radius,cref.density,'yo')
    ax[0][0].plot(radius1,density)
    ax[0][0].set_xlabel('Radius')
    ax[0][0].set_title('Density')
    
    
    ax[0][1].plot(ctrans.radius, ctrans.nu,'yo')
    ax[0][1].plot(radius1, nu)
    ax[0][1].set_xlabel('Radius')
    ax[0][1].set_title(r'$\nu$')
    
    ax[0][2].plot(ctrans.radius,ctrans.kappa,'yo')
    ax[0][2].plot(radius1,kappa)
    ax[0][2].set_xlabel('Radius')
    ax[0][2].set_title(r'$\kappa$')
        
    ax[1][1].plot(cref.radius,cref.temperature,'yo')
    ax[1][1].plot(radius1,temperature)
    ax[1][1].set_xlabel('Radius')
    ax[1][1].set_title('Temperature')
    
    ''''
    #Activate this if your case is magnetic
    ax[1][0].plot(ctrans.radius, ctrans.eta,'yo')
    ax[1][0].plot(radius1, eta)
    ax[1][0].set_xlabel('Radius')
    ax[1][0].set_title(r'$\eta$')
    '''
    
    ax[2][1].plot(cref.radius, cref.dsdr,'yo')
    ax[2][1].plot(radius1, dsdr)
    ax[2][1].set_xlabel('Radius')
    ax[2][1].set_title('Log entropy gradient')   
    
    ax[1][2].plot(cref.radius, cref.gravity,'yo')
    ax[1][2].plot(radius1, gravity*density/cp)
    ax[1][2].set_xlabel('Radius')
    ax[1][2].set_title('Gravity')   

    ax[2][0].plot(cref.radius, cref.heating,'yo')
    ax[2][0].plot(radius1, hprofile/density/temperature*lsun)
    ax[2][0].set_xlabel('Radius')
    ax[2][0].set_title('Heating')      
    
    plt.tight_layout()
    plt.show()
